In [ ]:
// If you are running this on your local machine, change the below file path
system "l /jqnotebooks/loaddata.q"

# Data Overview

The taxi database details cab fares in New York City. The data was provided from the
[NYC Taxi & Limousine Commission](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml).

The database we have loaded contains trip data starting at January 1st, 2009 until March 31st, 2009. The
database contains 6.9 million records and occupies 6Gb on disk with no compression. 

The taxi database is stored in a table called `trips`. To supplement the data, there is an additional New York City weather dataset called `weather`. This data was collected by the  [National Oceanic and Atmospheric Administration](https://www.noaa.gov) and distributed by the 
[National Climate and Data Center](https://www.ncdc.noaa.gov). The weather dataset we have loaded contains daily
summaries for January 2009 from 187 distinct weather stations surrounding New York City.

# Data Exploration

Tables form the core of Kx technology. Here we will go through how to access tables and examine their contents.

The first thing to do is to see what tables are currently in the database

In [ ]:
tables[]

we can simply view the data by using the table name

In [ ]:
smalltrips

It is very easy to interrogate the kdb+ process to get information from the process about the size and shape of the tables.

We can see how many records are in the table, by using [`count (code.kx)`](https://code.kx.com/v2/ref/count).

We can also inspect the schema of `trips` with [`meta (code.kx)`](https://code.kx.com/v2/ref/meta), obtaining a table of available columns and their types and other information. The following columns are produced:

- `c`: column name
- `t`: column type - [see code.kx.com](https://code.kx.com/v2/ref/#datatypes)
- `f`: foreign keys - [see code.kx.com](https://code.kx.com/q4m3/8_Tables/#85-foreign-keys-and-virtual-columns)
- `a`: attributes - any column modifiers applied for performance characteristics [see code.kx.com](https://code.kx.com/v2/basics/syntax/#attributes)

In [ ]:
count trips
meta trips

## qSQL

For those of you who have SQL experience, the syntax of writing queries in q will be very similar.

> Lots of areas of Q work to reduce the code required where it makes sense, in this case qSQL is like SQL but does not require the '*' to select all columns

In [ ]:
select from smalltrips

The following simple query extracts the `vendor`, `pickup_time` and `fare` columns from the entire table.

In [ ]:
select vendor, pickup_time, fare from smalltrips

As another example, the query below extracts a few columns, but also includes two filter criteria. Just as in SQL, table results can be filtered by expressions following a `where`. The following returns the number of taxi trips in the first day of data with 2 passengers.

Multiple filter criteria, separate by `,`, will be evaluated starting with the left-most criteria.

> **Note** the first day of data is extracted by applying `min` over the `date` column, and extracting all dates equal to that minumum month. Many other aggregations are included in kdb+ and will be introduced later.

In [ ]:
select date, month, vendor, passengers, fare, tip from trips 
    where date = min date, tip > 20   

New columns can be generated and added to the result by using assignment in the form `column_name : value`.

In [ ]:
select i, duration, total: fare + tip, fare, tip from trips

In addition to existing and computed columns, a virtual column `i` exists which maps to a records index within the table. A simple aggregation can be obtained by taking the count of this virtual column.

In [ ]:
select count i from trips where date = min date, passengers = 2

### Exercise 1: 
- How many trips were in the first date with less than two passengers?
- What about the last date?

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex1_a[]
ex1_b[]

### Exercise 2: 
- Select payment_type and fare from the trips data for the first date


In [ ]:
// Enter your qSQL code here 

In [ ]:
ex2_a[]

## Assignment

When exploring data, storing intermediate results is useful for avoiding repeated work. In q, [`: (code.kx)`](https://code.kx.com/v2/basics/syntax/#colon)  is used as assignment rather than `=`, since `=` is used for [equality](https://code.kx.com/v2/ref/equal) comparison. 

A new variable can be created with the form

```
var_name: expression
```

In order to take a closer look at only the data cooresponding to January 2009, the filtered table is stored in a variabled called `jan09`.

In [ ]:
jan09:select from trips where date within 2009.01.01 2009.01.31

count jan09

In [ ]:
select count i by date from jan09

> **Note** By making this assignment the new variable will be stored in memory no matter whether to source data was also in memory or stored on disk

### Exercise 3: 
- Assign the result of exercise 2 to a variable called res2


In [ ]:
// Enter your qSQL code here 

In [ ]:
// Excercise Answer to be deleted
res2:ex3_a[];
select count i by payment_type from res2

### Aggregations

With the filtered data in `jan09`, aggregations and other analytics can be run without performing the filter query again. Below is a simple aggregation returning the sum of both the `fare` and `tip` columns. 

In [ ]:
select sum fare, sum tip from jan09

[`sum`](https://code.kx.com/ref/v2/sum) is one of many built-in aggregations. Other built-in aggregations include, but are not limited to

- [`avg`](https://code.kx.com/v2/ref/avg) - average (mean)
- [`med`](https://code.kx.com/v2/ref/med) - median
- [`min`](https://code.kx.com/v2/ref/min) - minimum value
- [`max`](https://code.kx.com/v2/ref/max) - maximum value
- [`count`](https://code.kx.com/v2/ref/count) - number of values

### Exercise 4: 
- Try obtaining other aggregations on the fare and tip columns.


In [ ]:
// Enter your qSQL code here 

In [ ]:
ex4_a[]

### Grouping with `by` 

Unlike SQL, qSQL allows the developer to perform grouping and aggregation seperately.
The easiest way to group similar values together is to use the `by` clause.

When used without an aggregation, `by` will return a list of values from the selected column. Since list types are completely valid types in Q this type of thing is permissable

In [ ]:
select fare by vendor from jan09

In order to extract some useful insight from the data, grouping is best used with aggregation functions as shown below:

In [ ]:
select sum fare, sum tip by vendor from jan09

### Exercise 5: 
- What is the biggest tip for each company?

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex5_a[]

### Exercise 6: 
- What payment_type generates the highest tips?

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex6_a[]

### Using `fby` to avoid nested queries

Commonly in SQL queries, nested queries are required in cases where filter criteria require aggregations in the context of some other column. For example, asking for all records where the taxi ride duration is greater than the average duration for that taxi's vendor can be simply expressed using `fby`.

The form of fby is `(aggregation;data) fby group`, as in the below query.

In [ ]:
select from jan09 where duration < (avg;duration) fby vendor

Compare the above statement to the how it would be similarly done via normal qSql commands, we would first get the avg duration for each vendor and then perfrom a qsql query for each duration and vendor:

In [ ]:
resby:select avg duration by vendor from jan09
raze {select max fare by vendor from jan09 where vendor = x[`vendor], duration < x[`duration]} each 0!resby

You can see how simpiler an fby clause is compared to the above statements

### Exercise 7: 

- Which payment type produces the highest average tip when only trips with a fare larger than the average for each vendor is considered?
- Which vendor has the largest number of trips when only considering trips smaller than the average duration for each vendor?

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex7_a[]

In [ ]:
ex7_b[]

### Updating existing data

When exploring data, fixing bad data is a common task, whether than be from incorrect data entry, incompatible units of measure, or other reasons. 

If we inspect the data for maximum passengers for each vendor, the maximum number for 2 vendors is 6.

In [ ]:
select max passengers by vendor from jan09

If the above data is decided to be wrong, it can be easily fixed using `update` to change the data. In the below, the passengers column is set to the value `5` in any entry where the passenger count is greater than 5. This is then assigned back to `jan09` to hold on to the result of changing the data.

In [ ]:
select from jan09 where passengers > 5

In [ ]:
jan09: update passengers: 5 from jan09 where passengers > 5

In [ ]:
jan09

Observe below that the maximum passengers reported has now been updated.

In [ ]:
select max passengers by vendor from jan09

### Temporal Arithmetic

Kdb+ natively supporty several temporal types, and arithmetic between these types. This makes writing queries involving time especially simple. The `pickup_time` column in the data is of the *timestamp* type. 

[`Dataypes (code.kx)`](https://code.kx.com/v2/ref/#datatypes)

As an example, we could convert the `pickup_time` values to their `minute` value (including hours and minutes), and group the data based on this time frame.

> **Note**
The below query returns the total money made each minute by cabs in January. Observe the following expression within the query below:
```
sum fare + tip
```
This expression highlights a unique aspect of q: **evaluation is from right-to-left**. `fare` and `tip` are added first, producing the totals for each trip, then these totals are processed by `sum` producing the aggregation.

In [ ]:
select total:sum fare + tip by pickup_time.minute from jan09

In [ ]:
select total:(sum fare) + tip by pickup_time.minute from jan09

We could further aggregate on time by using `xbar` to bucket the minutes into hours. Grouping the minutes into 60-unit buckets would produce hours, as expressed below.

In [ ]:
select count i by 60 xbar pickup_time.minute from jan09 where date = 2009.01.01

### Excercise 8:
- Show the largest tip for each 15 minute timespan during the month of january

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex8_a[]

 - Now break up this information down by vendor

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex8_b[]

## Joins

A join combines data from two tables, or from a table and a dictionary.

Some joins are keyed, in that columns in the first argument are matched with the key columns of the second argument.

Some joins are as-of, where a time column in the first argument specifies corresponding intervals in a time column of the second argument. Such joins are not keyed.

In each case, the result has the merge of columns from both arguments. Where necessary, rows are filled with nulls or zeroes. 

Here is a list of some joins possible using q/kdb+:

- [Left Join](https://code.kx.com/v2/ref/lj/)
- [Inner Join](https://code.kx.com/v2/ref/ij/)
- [Plus Join](https://code.kx.com/v2/ref/pj/)
- [AJ (As-of) Join](https://code.kx.com/v2/ref/aj/)
- [Window Join](https://code.kx.com/v2/ref/wj/)

In order to demonstate some common join types we will use data from preloaded table, `weather`,  which corresponds to the same time as our taxi data. In this case we will load some weather data and try and get some insight in how this affected taxi journies around this period.

In [ ]:
select mindate:min date, maxdate:max date from weather

### Exercise 9:
- Display the max and min tempratures for NYC each week through Janurary (for this query a week is just every 7 days)

In [ ]:
// Enter your qSQL code here 

In [ ]:
// Excerice Answer
ex9_a[]

We now have 2 tables of related data, taxi trips and weather data for each day on which a trip ocurred.

It would be nice if we could combine these tables so that we casn easily ask questions across both data sets.

For example, are average trip durations shorter or longer on days with lots of precipitation?

Like other SQLs, q-sql supports a number of join operations. Here we will use a left join to create a single table containing both trip and weather data. In q-sql the `lj` operator requires one or more common columns between the tables to join across. 

For our purpose we will be looking at the daily weather data so we'll use the `date` column:

In [ ]:
jan09C:select trips: count i by date from jan09
jan09C

### Left Join

`t1 lj t2` - [code.kx - left join](https://code.kx.com/v2/ref/lj/)

We now have a date column in both tables. 

The `lj` operator requires that at least the right hand table argument be keyed. A table can be keyed in a number of ways, however the easiest is to use the `xkey` function


In [ ]:
jan09W:(0!jan09C) lj `date xkey select date, precip from weather 
jan09W

Now can look at trips vs precipitation

In [ ]:
select date,trips,precip from jan09W

## Excercise 10:
- Create a new join which joins the number of trips with the average temprature from the weather data

In [ ]:
// Enter your qSQL code here 

In [ ]:
// Excerice Answer
ex10_a[]

### Inner Join

`t1 lj t2` - [code.kx - inner join](https://code.kx.com/v2/ref/ij/)

The `ij` operator acts in the same way as the left join but only records found in the both tables will remain in the resultant table. See below when the date, 2009.01.03 is deleted from weather.

In [ ]:
jan09WInner:(0!jan09C) ij `date xkey select date, precip from weather where date <> 2009.01.03
select date,trips,precip from jan09WInner

You will notice that the date of 2009.01.03 is not found in the resultant table output when compared to jan09W.

### Plus Join

`t1 pj t2` - [code.kx - plus join](https://code.kx.com/v2/ref/pj/)

The plus-join, `pj`, is another powerful join in q-sql. The plus join is simlar to `lj`, but will sum the non-key columns

In [ ]:
select sum fare by vendor from jan09

We can use the plus join to sum two vendors fares together, we will take look at how to do this below:

In [ ]:
vtsfares:select tripcount:count i,sum fare,sum tip by date from jan09 where vendor = `VTS
cmtfares:select tripcount:count i,sum fare,sum tip by date from jan09 where vendor = `CMT

We have now generated two tables for two seperate vendors, the output for each below:

In [ ]:
vtsfares
cmtfares

If we perform a plus join in these two simiar tables, we can find out the total fares obtained between the two vendors:

In [ ]:
vtsfares pj cmtfares

### AJ (asof) Join

`aj[matching columns;t1;t2]` - [code.kx - aj join](https://code.kx.com/v2/ref/aj/)

qSQL also supports time-series joins, a powerful feature typically not found in other databases and languages.

Given the data we have, we could ask what were the latest pick-ups for each vendor, `aj` a particular time.

We will create a temporary time table with a minimun date time for each vendor:

In [ ]:
timetab:([] vendor: `VTS`DDS`CMT; pickup_time:3#2009.01.06D03:30:00)
timetab

We will then perform an aj join to get the latest information for each vendor before the time in timetab

In [ ]:
aj[`vendor`pickup_time;timetab;jan09]

The result is the record for each vendor with the pickup_time <= to the time we specified
- An `aj` join will alway select the last record before the specificed time.

In [ ]:
timetab:([] vendor: `VTS`DDS`CMT`VTS`DDS`CMT; pickup_time: ((3# 2009.01.06D03:30:00),3#2009.01.06D04:30:00))
timetab

We have created a new timetab table, what will the output asof join be now?

In [ ]:
aj[`vendor`pickup_time;timetab;jan09]

### Window Joins

`wj[window;columns;t1;(t2;(col1;function1);(col2;function2))]` - [code.kx - window join](https://code.kx.com/v2/ref/wj/)

Another popular time series join is the window join, `wj`. Window join is a generalization of as-of joins, and are designed to aggregate values of specified columns within certain intervals. 

The window join is similar to an asof join with 2 main differences:-
- First, we specify one or more windows to join over
- Second, we can supply aggregation functions to apply over the window

[Transforming time to various different time formats](https://code.kx.com/q4m3/7_Transforming_Data/#723-casts-that-narrow)

We will work below to find the average tip and the average number of passengers every 30 minutes for the vendor CMT. First we will create a table to get the count of the number of rides in 30 minutes buckets for 2009.01.01. 

In [ ]:
thirtyBuckets:0!select count i by 30 xbar pickup_time.minute, vendor from jan09 where date = 2009.01.01, vendor = `CMT;
4#thirtyBuckets

We will then create a new temporary table and create a minute column:

In [ ]:
tmpt:select vendor, pickup_time.minute, passengers, tip from jan09 where date = 2009.01.01, vendor = `CMT;
4#tmpt

We then need to create window periods for the window join

In [ ]:
w:exec distinct minute from thirtyBuckets
w:-29 0+\: w
w

And finally we can create your window join:

In [ ]:
wj1[w;`vendor`minute;thirtyBuckets;(tmpt;(max;`passengers);(avg;`tip))]

# Lists

[code.kx - q4m - lists](https://code.kx.com/q4m3/3_Lists/)

So far we've seen how a _table_ is a natural fit for storing and analysing huge amounts of data. Under the covers though, kdb+ leverages a more fundamental data structure to build the table: the _list_. A large part of the performance edge that kdb+ comes from its ability to work directly with lists - every user should be familiar with them!

To get our hands on our first list, consider the `trips` table that we've already met. If all you cared about is the amount people paid for their journey, you've already seen that we can `select` out just the column of interest (with an optional `where` clause)

In [ ]:
vtsfares:select fare from trips where date = 2009.01.01, vendor=`VTS
vtsfares

Since tables in kdb+ are *column-oriented*, columns can be extracted from the table by simpling indexing into the table with the column name, producing a contiguous vector, or *list*, representing that column. 

In [ ]:
fares: vtsfares`fare
fares

In [ ]:
type fares

Operations on vectors work just as they do within `select` queries, as shown below. There is often no need for loops or other constructs for processing lists due to this design, making processing data and writing custom analytics much simpler even outside of `select` queries. 

In [ ]:
avg fares

In [ ]:
select avg fares from vtsfares

To check that we've got what we expect, we can use the `type` operator:

In [ ]:
type fares

Since the number is positive, and less than 20, we can tell that we have a _simple list_. 
In a simple list, the types of the individual elements are all the same. In contrast, you'll often see _general lists_, which have `type` 0, and have elements of more than one different `type`.

In [ ]:
generalList:(`a;1i)
type generalList

A list can be _empty_ - if there had been a typo in the select statement, and we inquired about a non-existent cab company, we'd see:

In [ ]:
svtfares:select fare from trips where month=2009.01m, vendor=`SVT
svtfares`fare

Thankfully, we made no such mistake - we've plenty of records to work with, as we can verify with the `count` builtin:

In [ ]:
count fares

We can look at the results just by passing the variable name `fares`:

In [ ]:
fares

Yikes! That's a lot of screenspace to waste. 
To save the electrons, we can just look at the first few elements using the `sublist` operator:

In [ ]:
10 sublist fares

sublist is clever - to get the _last_ few elements, all you have to do is give a negative number:

In [ ]:
-10 sublist fares

## Exercise 11:
- Use sublist to get the the second 10 elements in the list

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex11_a[]

Sublist has a nice property that the number of elements it returns is _capped_ at the size of the list that it operates on:

In [ ]:
count 10000000 # fares

Let's put a bit of structure on our list - let's make a _sorted_ copy of it that we can play with. 
kdb+ offers the operator `asc` to do this:

In [ ]:
sortedFares:asc fares

`sortedFares` has the same `count` and `type` as fares, but now is _sorted_ in ascending order. If you looked only at the first elements of this list, you might conclude that cab journeys in NYC are great value!

In [ ]:
10 sublist sortedFares

## Exercise 12:
- Use sublist to obtain the 10 highest values from the sorted List

In [ ]:
// Enter your qSQL code here 

In [ ]:
// Exercise Answer
ex12_a[]

### Obtaining Randon Data

We've fallen into an easy trap to fall into - we've extracted an _unrepresentative sample_. 
To pick 10 _random_ records from the list, we can use the `?`operator:

[code.kx - ? - Roll,Deal](https://code.kx.com/v2/ref/deal/)

In [ ]:
sampleFares:10?sortedFares;
sampleFares

That's more like it!

`reverse` can be used to get a list where the first element has been swapped with the last and so on:

In [ ]:
10 sublist reverse sortedFares

Lists support _random access_. To pick out the 10th element of a list, we use:

In [ ]:
fares[10]

The preceding yields an atom, as can be verified by using `type`. A similar approach works for lists of indexes:

In [ ]:
fares[0 1 2 3 4 5 6 7 8 9]

(Normally, you'd use a builtin, `til`, to get the list of the first N ints, starting at zero: as you've seen, q uses zero indexing)

[code.kx - til ](https://code.kx.com/v2/ref/til/)

In [ ]:
til 10
fares[til 10]

## Exercise 13:
- Extract the 10th to the 20th elements from the fares list using the til keyword

In [ ]:
// Enter your qSQL code here

In [ ]:
// Exercise Answer
ex13_a[]

## Exercise 14:
- Use Indexing to find the median value in the sortedFares list. 

In [ ]:
// Eneter your Q Code here

In [ ]:
// Exercise Answer
ex14_a[]

In the case of a simple list, if the index used is too high, a _null_ of the appropriate type is returned.

In [ ]:
fares[count fares]
fares[-1+count fares]

It's a common task to have to get the element at the _head_ of a list or at the _tail_ of a list. 
This is easily acheived by the `first` and `last` operators respectively.

In [ ]:
first sortedFares
last sortedFares

The below code block will also obtain the last value from the list: 

In [ ]:
((count sortedFares)-1)_sortedFares

> take note that there is a difference between what is returned by `1 sublist sortedFares` and `first sortedFares`. The first returns a list of one element and the second is returning an atom. You can see below how Q represents each on the console

In [ ]:
1 sublist sortedFares
first sortedFares

> Be careful when viewing lists on the console,  2D lists have an output in a similar format to atoms

In [ ]:
(),(enlist 499)
(),499

enlist 499 
enlist (enlist 499)

### List Amendment
A simple list can be indexed into using the `@` operator:

In [ ]:
2* til 5
@[sampleFares;(2*til 5)]

The `@` parameter can also have additional parameters applied to it so that the list can be altered, below we are assinging the positions from `2*til 5` to be 99f instead:

In [ ]:
@[sampleFares;(2*til 5);:;99f]

or we can use `+` instead of `:` to add 99f to the current values

In [ ]:
@[sampleFares;(2*til 5);+;99f]

The above is not a persistent change - it will make a copy of the fares list with a single value changed and output it to the terminal but the result is not saved 

In [ ]:
sampleFares

To persist the change, the name of the list is preceded by a backtick or by assigning the output to a new variable:

In [ ]:
test:@[fares;(2*til 4);:;0Nf]
test
@[`fares;(2*til 4);:;0Nf]
fares

A list can be grown by appending to the back of it by using the join operater `,`:

In [ ]:
fares,:12.34

### Operators on lists

So we've seen that we can get our hands on lists, can index into them to sample, and can mutate them.
We can also interrogate them with a range of operators that are optimized to work on list types. 
We've already met asc, which will make a sorted copy of your data. 
If you are only interested in the most someone paid for their cab ride, there's no need to sort. Just use the max operator!

In [ ]:
max[fares]

The builtins `sum`, `min`, `avg`, `dev`, `var` act in a similar way - they can operate directly on lists and return the result as an atom.

- [`sum`](https://code.kx.com/v2/ref/sum) - sum of the list
- [`min`](https://code.kx.com/v2/ref/min) - minimum value
- [`avg`](https://code.kx.com/v2/ref/avg) - average value
- [`dev`](https://code.kx.com/v2/ref/dev) - standard deviation value
- [`var`](https://code.kx.com/v2/ref/var) - Variance

In [ ]:
sum[fares]

Plenty of cash coming in for VTS!

## Exercise 15:
- What other aggregations can you reuse on fares.

In [ ]:
// Enter your qSQL code here 

In [ ]:
// Exercise Answer
ex15_a[]
ex15_b[]

Let's say that we wanted to find out the amount in Canadian dollars that each trip cost. 
To do this, we can just multiple the fare by the exchange rate:

In [ ]:
USDCAD:1.33
fares*USDCAD

If we wanted to map the fares paid into the range [0,1] we can just chain a few of the operators that we've already met to get:

In [ ]:
100*(fares-min[fares])%max fares

Perhaps we've had some data loss, or some otherwise dirty data. kdb permits null values to be used. Is this a problem for us?

In [ ]:
any null fares

This is exactly equivalent to using `any[null[fares]]` - but perhaps a little cleaner? Your mileage may vary!

We can also use the function `null` fares to find and locate instances where this is the case:

In [ ]:
where null fares

And we can pass this into the @ operater to replace this null with a new distinct value instead:

In [ ]:
@[fares;where null fares;:;-1f]

## Exercise 16:
- EUR / USD Exchange rate is 1.14. What is the maximum Fare in Euros?

In [ ]:
// Enter your qSQL code here 

In [ ]:
// Exercise Answer
ex16_a[]

### Cut

Elements can be dropped from the front a list using the `_` operator (known as drop)
`N _ fares`

In a similar fashion to sublist, a negative argument indicates that the last N elements of a list should be dropped.

In [ ]:
count fares
6848550 _ fares

In [ ]:
((count sortedFares)-1)_sortedFares
last sortedFares

A list can be reshaped into a 2 dimensional structure using the "cut" operator:

In [ ]:
first 10 cut fares

Returns a list of lists, where each row has count 10 (with the exception of that last row, which may have fewer).

The # operator, known as "take" performs a similar function to sublist. 
The crucial difference here is that # allows you to extract more elements than exist in the list.
It does this by padding:

In [ ]:
20#10 sublist fares

In [ ]:
(5 2)# 10 sublist fares

If a pair of values is specified as the first argument to #, a two dimensional rather than one dimensional is returned:


We've talked a lot about simple and numerical lists, but kdb is not restricted to these.
When there are elements of more than one single type in a list, it is referred to as a general list. 
For example, a pair representing the taxi company and the fare paid could look like:

In [ ]:
general:(`VTS;23.45);
general

While simple lists always have strictly positive values returned by type, general lists always have type zero.

In [ ]:
type general

joining entities of different types with the comma operator will produce a mixed list

In [ ]:
general:2018.01.01,102,`hello,enlist "world"
general
type general

## Functions


### Calling functions

Functions are called with the arguments in square brackets `[]`. For example we can call the builtin `max` function on a list like so.

In [ ]:
max[10 11 12]

When we have a unary (single parameter) function, we can omit the square brackets. These two lines are equivalent

In [ ]:
max[10 11 12]
max 10 11 12

### Defining functions

We can define our own functions in q, here is an example of a binary (two parameters) function which calculates the speed in km/h from the distance travelled in miles and the duration of a journey in hours.

In [ ]:
speed:{[miles;hours]
 mph:miles%hours;
 kph:1.609*mph;
 / return the speed in kph
 :kph;
 }

Here we refer to `miles` and `hours` as parameters of the `speed` function, `kph` is a local variable we define inside the function, which we then explicitly return from the function using `:`.

Functions can be defined over multiple lines, but _note_ that each line of a function after the first one must start with at least one whitespace character.

When we have more than one parameter the arguments to a function are separated by `;`. So an example call to the `speed` function looks like this.

In [ ]:
speed[15;0.5]

If there is no explicit return from a function using `:` then the result of the function is the last expression in it. So the code above can be rewritten as 

In [ ]:
speed:{[miles;hours]
 1.609*miles%hours / NOTE NO SEMICOLON
 }
speed[15;.5]

We can also call this function with a list of distances and a corresponding list of durations.

In [ ]:
speed[15 30 20;.5 .9 1.0]

#### Implicit parameters

When functions have up to three parameters, the parameter names can be omitted, with `x`,`y`, and `z` as implicit parameters. So our `speed` function can also be written as

In [ ]:
speed:{1.609*x%y}

Here we calculate the speed for two distances for a single duration.

In [ ]:
speed[15 30;.5] 

### Call functions from qSQL

So far we've only be calling functions on lists and atoms. But we can also call functions from qSQL.


In [ ]:
select speed:speed[distance;duration % 0D01:00],distance,duration from jan09 where vendor = `VTS

This gives us the speed in km/h for each trip in our `jan09` table. We use `duration % 0D01:00` to give us a number of hours as a floating point number from the nanosecond precision duration we have stored in the trips table.

We can combine functions we've defined, builtin functions from q, and qsql with grouping and aggregation to get an average speed for each vendor in our `jan09` table.

In [ ]:
select avgspeed:speed[sum distance;sum[duration]%0D01:00] by vendor from jan09

### Use qSQL inside functions

QSQL queries can be called just like any other q code inside functions, One very common use of this is to build parameterized queries.



In [ ]:
top10:{[vnd]
 10 sublist `fare xdesc select from jan09 where vendor=vnd
 }
 

This gives us the top 10 trips by fare value for any vendor.

In [ ]:
top10 `CMT

So what happens when we we have a list of vendors and we want the top 10 trips for each vendor? We can get that result simply by applying our function to each of a list of vendors using the `each` keyword. `each` is an example of what we call an *adverb* in q, these modify the behaviour of existing functions.

In [ ]:
besttrips:top10 each `CMT`CND
besttrips

That gives us a list of tables, 1 for each vendor we asked for, we can put these all together using the `raze` q function.

In [ ]:
raze besttrips

You might have noticed above that we called our function top10, what if we wanted the top 20 or top 30 trips for each vendor? Instead of hardcoding the value 10 we can make this a parameter of a `topN` function like this.

In [ ]:
topN:{[N;vnd]
 N sublist `fare xdesc select from jan09 where vendor=vnd
 }

Now we can call this function with 10 as a parameter, or if we still want a top10 function we can derive this function from our topN function using something called a *projection*. A projection is a function with one-or-more of the parameters fixed with specific value(s).

In [ ]:
topN[10;`CMT] / call with both parameters
top10:topN[10;] / derive a projection which calculates the top10

Projections and functions are called in the same way so we can still call topN or top10 for each of a list of vendors. The two results below are the same.

In [ ]:
raze top10 each `CMT`VNT
raze topN[10;]each `CMT`VNT

And we can use the `~` operator to compare if the above two tables are the exact same:

In [ ]:
(raze top10 each `CMT`VNT) ~ raze topN[10;]each `CMT`VN

### Exercise 17:
- Create a Function that takes two inputs and has input x divided by y
- Create a another function that will select from jan09 the columns vendor, distance, tip and a new column that takes the output of the function above using the columns tip & distance
- Find out the average tip per mile per vendor from the second function you have created

In [ ]:
//Enter your Q code here

In [ ]:
ex17_a[]
ex17_b[]
ex17_c[]